In [1]:
import wget
import pandas as pd
from nltk import WordPunctTokenizer
from collections import defaultdict, Counter
import numpy as np
import random

In [2]:
file_names = ['Арап Петра Великого.txt', 'Дубровский.txt', 'Египетские ночи.txt', 'История Пугачёва.txt',
              'История села Горюхина.txt', 'Капитанская дочка.txt', 'Кирджали.txt', 'Пиковая дама.txt',
              'Повести Белкина.txt', 'Рославлев.txt']

In [3]:
for file_name in file_names:
    wget.download(f'https://raw.githubusercontent.com/d0rj/RusLit/main/prose/Pushkin/{file_name}')

In [4]:
data = pd.DataFrame()
for file_name in file_names:
    with open(file_name, 'r', encoding='utf-8') as file:
        text = file.read()

        df = pd.DataFrame({
            file_name.replace('.txt', ''): text
        }.items(), columns=['name', 'text'])
        data = pd.concat([data, df], ignore_index=True)

In [5]:
data

,name,text
0,Арап Петра Великого,Александр Пушкин\nАРАП ПЕТРА ВЕЛИКОГО\nГЛАВА I...
1,Дубровский,\nАлександр Сергеевич Пушкин. Дубровский\n\n\n...
2,Египетские ночи,Александр Пушкин\nЕГИПЕТСКИЕ НОЧИ\nГЛАВА I\n— ...
3,История Пугачёва,\nАлександр Сергеевич Пушкин\nИстория Пугачева...
4,История села Горюхина,Александр Пушкин\nИСТОРИЯ СЕЛА ГОРЮХИНА\nЕсли ...
5,Капитанская дочка,Александр Пушкин\nКАПИТАНСКАЯ ДОЧКА\nБереги че...
6,Кирджали,Александр Пушкин\nКИРДЖАЛИ\nПОВЕСТЬ\nКирджали ...
7,Пиковая дама,Александр Пушкин\nПИКОВАЯ ДАМА\nПиковая дама о...
8,Повести Белкина,\nПовести покойного Ивана Петровича Белкина\n\...
9,Рославлев,Александр Пушкин\nРОСЛАВЛЕВ\nЧитая «Рославлева...


In [6]:
tokenizer = WordPunctTokenizer()

In [7]:
def get_tokens(text):
    return tokenizer.tokenize(text.lower())

In [8]:
text = ' '.join(data['text'].tolist())

In [9]:
tokens = get_tokens(text)

In [10]:
tokens[:10]

['александр',
 'пушкин',
 'арап',
 'петра',
 'великого',
 'глава',
 'i',
 'я',
 'в',
 'париже']

In [11]:
def generate_trigrams(tokens):
    trigrams = []
    for i in range(len(tokens) - 2):
        trigrams.append((tokens[i], tokens[i + 1], tokens[i + 2]))
    
    return trigrams

In [12]:
trigrams = generate_trigrams(tokens)

In [13]:
trigrams[2]

('арап', 'петра', 'великого')

In [14]:
model = defaultdict(Counter)

In [15]:
for w1, w2, w3 in trigrams:
    model[(w1, w2)][w3] += 1

In [16]:
model

defaultdict(collections.Counter,
            {('александр',
              'пушкин'): Counter({'арап': 1,
                      'египетские': 1,
                      'история': 1,
                      'капитанская': 1,
                      'кирджали': 1,
                      'пиковая': 1,
                      'рославлев': 1}),
             ('пушкин', 'арап'): Counter({'петра': 1}),
             ('арап', 'петра'): Counter({'великого': 1}),
             ('петра',
              'великого'): Counter({'глава': 1, '.': 1, 'они': 1, ',': 1}),
             ('великого', 'глава'): Counter({'i': 1}),
             ('глава', 'i'): Counter({'я': 1, '.': 1, '—': 1, 'сержант': 1}),
             ('i', 'я'): Counter({'в': 1}),
             ('я',
              'в'): Counter({'париже': 1,
                      'петербурге': 1,
                      'театр': 1,
                      'симбирск': 1,
                      'недоумении': 1,
                      'ней': 2,
                      'бешенстве': 

In [17]:
def generate_text(model, seed_text, num_words=50):
    text = seed_text.split()

    if len(text) < 2:
        while len(text) != 2:
            text.append(random.choice(list(model.keys()))[0])

    while len(text) < num_words:        
        w1, w2 = text[-2].lower(), text[-1].lower()
        
        # Получаем вероятные продолжения (слово3)
        next_words = model.get((w1, w2))

        if next_words:
            words = list(next_words.keys())
            words_count = list(next_words.values())

            next_word = words[np.argmax(words_count)]

            text.append(next_word)
        else:
            text.append(random.choice(list(model.keys()))[0])
        
    return ' '.join(text)



In [18]:
seed_text = 'я император'

In [19]:
generated_text = generate_text(model, seed_text, num_words=50)
generated_text

'я император петрович шибко прочие несли стащили тебе - приходили жестокое у вдруг благопристойно греха величайшим гренадеры татищевой к главе первой 1 некоторые из его шайки . он был в таком случае , что он не мог не улыбнуться . императрица , досадуя на сплетни европейские , отвечала с какою'

In [22]:
seed_text = ''

In [23]:
generated_text = generate_text(model, seed_text, num_words=50)
generated_text

'вольность молодую тройкою оной вальсы кармана федоровым дубровский всех бегут . караулами давыдовым оскорбительных они и очистят воровскую рощу ; народ не трусливый , каждый в одиночку на медведя ходит – от разбойников не попятятся . – а что нам делать , коли не наша воля ? — спросил я'